In [1]:
pip install scikit-surprise

Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp39-cp39-macosx_10_9_universal2.whl size=1016133 sha256=e05caf958d161ece2c2289d43fdddd92379b87351fe3befaac947342124806a1
  Stored in directory: /Users/steven.souksavath/Library/Caches/pip/wheels/42/41/d3/a56ae864ad22cc6583ec9312be43fbc611c87e53dc49aac953
Successfully built scikit-surprise
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from surprise import Dataset, Reader
from surprise import KNNBasic
from surprise.model_selection import cross_validate

In [5]:
ratings_df = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1/main/Datasets/ratings.csv')
movies_df = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1/main/Datasets/movies.csv')

In [6]:
data = pd.merge(ratings_df, movies_df, on='movieId')

In [9]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(data[['userId', 'movieId', 'rating']], reader)

In [10]:
sim_options = {'name': 'cosine',
                'user_based': False} #content based filtering

In [11]:
algo = KNNBasic(sim_options=sim_options)

In [12]:
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9670  0.9733  0.9777  0.9745  0.9803  0.9746  0.0045  
Fit time          2.56    1.99    2.03    1.70    1.84    2.02    0.29    
Test time         3.28    3.42    3.23    3.35    3.26    3.31    0.07    


{'test_rmse': array([0.9669901 , 0.97332106, 0.97767183, 0.9744617 , 0.98033168]),
 'fit_time': (2.5630252361297607,
  1.9918501377105713,
  2.0271313190460205,
  1.6979470252990723,
  1.8371238708496094),
 'test_time': (3.2811436653137207,
  3.4207170009613037,
  3.230088949203491,
  3.3474481105804443,
  3.25663685798645)}

In [13]:
algo.fit(data.build_full_trainset())

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [14]:
print(algo.sim)

[[1.         0.90773108 0.96657344 ... 1.         1.         1.        ]
 [0.90773108 1.         0.88581521 ... 1.         1.         1.        ]
 [0.96657344 0.88581521 1.         ... 1.         1.         1.        ]
 ...
 [1.         1.         1.         ... 1.         1.         1.        ]
 [1.         1.         1.         ... 1.         1.         1.        ]
 [1.         1.         1.         ... 1.         1.         1.        ]]


In [15]:
data

In [18]:
df = pd.merge(ratings_df, movies_df, on='movieId')

In [17]:
pred = algo.predict(uid=1, iid=10)
pred.est

3.9880309408033616

In [19]:
df[df['userId']==1]

,userId,movieId,rating,timestamp,title,genres
0,1,16,4.0,1217897793,Casino (1995),Crime|Drama
1,1,24,1.5,1217895807,Powder (1995),Drama|Sci-Fi
2,1,32,4.0,1217896246,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,4.0,1217896556,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,4.0,1217896523,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
...,...,...,...,...,...,...
108,1,48780,4.0,1217897632,"Prestige, The (2006)",Drama|Mystery|Sci-Fi|Thriller
109,1,49272,3.5,1217896576,Casino Royale (2006),Action|Adventure|Thriller
110,1,52973,3.5,1217897820,Knocked Up (2007),Comedy|Drama|Romance
111,1,54286,4.0,1217896447,"Bourne Ultimatum, The (2007)",Action|Crime|Thriller


In [33]:
# This is how an item based filtering works

df[(df['userId']== 30) & (df['movieId'] == 33794)]

,userId,movieId,rating,timestamp,title,genres
2827,30,33794,5.0,1314480601,Batman Begins (2005),Action|Crime|IMAX


In [ ]:
similarities = algo.sim

sns.heatmap(np.array(similarities), annot=True)

In [26]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.25)

In [27]:
algo = KNNBasic(sim_options={'user_based': True})

In [28]:
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [35]:
#This is the first row for the predictions below (True value for the user)

df[(df['userId']== 15) & (df['movieId'] == 2644)]

,userId,movieId,rating,timestamp,title,genres
1020,15,2644,5.0,1447478260,Dracula (1931),Horror


In [29]:
#The model tries to predict the above user and other user ratings

predictions = algo.test(testset)
predictions

[Prediction(uid=15, iid=2644, r_ui=5.0, est=4.011345362878378, details={'actual_k': 8, 'was_impossible': False}),
 Prediction(uid=232, iid=79293, r_ui=4.5, est=3.249979030217948, details={'actual_k': 13, 'was_impossible': False}),
 Prediction(uid=668, iid=45666, r_ui=1.5, est=3.0121471987902533, details={'actual_k': 4, 'was_impossible': False}),
 Prediction(uid=546, iid=4970, r_ui=4.0, est=4.169045494070329, details={'actual_k': 4, 'was_impossible': False}),
 Prediction(uid=370, iid=150, r_ui=5.0, est=4.140522571254396, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=531, iid=364, r_ui=5.0, est=4.233274174030994, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=545, iid=150, r_ui=3.0, est=3.8894049735526006, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=461, iid=3108, r_ui=1.0, est=3.5964167609641704, details={'actual_k': 32, 'was_impossible': False}),
 Prediction(uid=128, iid=1603, r_ui=2.5, est=2.7332212019374027, details={

In [36]:
df[df['userId']==1]['movieId'].unique()

array([   16,    24,    32,    47,    50,   110,   150,   161,   165,
         204,   223,   256,   260,   261,   277,   296,   318,   349,
         356,   377,   380,   457,   480,   527,   589,   590,   592,
         593,   597,   608,   648,   719,   724,   736,   780,   858,
         912,   968,  1061,  1089,  1136,  1196,  1198,  1210,  1213,
        1220,  1221,  1222,  1233,  1243,  1258,  1265,  1267,  1270,
        1287,  1580,  1617,  1721,  1923,  1961,  2021,  2028,  2105,
        2161,  2194,  2396,  2407,  2455,  2467,  2502,  2542,  2571,
        2628,  2716,  2728,  2762,  2858,  2947,  2959,  3256,  3421,
        3578,  4011,  4027,  4033,  4085,  4262,  4306,  4963,  4993,
        4995,  5349,  5378,  5418,  5445,  5952,  6365,  6711,  6807,
        7153,  8825,  8961, 32587, 33493, 33794, 33834, 45950, 48516,
       48780, 49272, 52973, 54286, 57949])

In [37]:
algo.predict(uid=1, iid=10)

Prediction(uid=1, iid=10, r_ui=None, est=3.469836004571536, details={'actual_k': 40, 'was_impossible': False})

In [38]:
df[df['movieId']==10]

,userId,movieId,rating,timestamp,title,genres
468,7,10,4.0,1322062970,GoldenEye (1995),Action|Adventure|Thriller
690,9,10,3.0,842686329,GoldenEye (1995),Action|Adventure|Thriller
833,11,10,3.0,850816421,GoldenEye (1995),Action|Adventure|Thriller
2390,28,10,3.0,884100808,GoldenEye (1995),Action|Adventure|Thriller
2529,29,10,4.0,836583421,GoldenEye (1995),Action|Adventure|Thriller
...,...,...,...,...,...,...
94884,627,10,3.5,1237934561,GoldenEye (1995),Action|Adventure|Thriller
97622,650,10,5.0,965517712,GoldenEye (1995),Action|Adventure|Thriller
98091,653,10,4.0,829071627,GoldenEye (1995),Action|Adventure|Thriller
98262,656,10,4.0,838777501,GoldenEye (1995),Action|Adventure|Thriller


# Single Value Decomposition (SVD)

In [39]:
from surprise import SVD

In [41]:
svd = SVD()

#Test if this model works for the data
cross_validate(svd, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8684  0.8739  0.8754  0.8667  0.8628  0.8694  0.0047  
Fit time          0.62    0.64    0.63    0.64    0.62    0.63    0.01    
Test time         0.05    0.05    0.05    0.05    0.14    0.07    0.03    


{'test_rmse': array([0.86841296, 0.87391145, 0.87539579, 0.86671164, 0.86275647]),
 'fit_time': (0.62294602394104,
  0.6350448131561279,
  0.6335978507995605,
  0.6435420513153076,
  0.6229448318481445),
 'test_time': (0.053476810455322266,
  0.05135321617126465,
  0.05178499221801758,
  0.05488300323486328,
  0.1367509365081787)}

In [42]:
trainset2 = data.build_full_trainset()
svd.fit(trainset)

In [43]:
svd.predict(1, 10)

Prediction(uid=1, iid=10, r_ui=None, est=3.6126890258417137, details={'was_impossible': False})

In [44]:
#predictions for everyone to compare to what actually happened - TAKENOTE PRETTY USEFUL for the below to see model performance and accuracy

df['predicted_rating'] = df.apply(lambda x: svd.predict(x['userId'], x['movieId']).est, axis=1)

In [45]:
df

,userId,movieId,rating,timestamp,title,genres,predicted_rating
0,1,16,4.0,1217897793,Casino (1995),Crime|Drama,3.999100
1,1,24,1.5,1217895807,Powder (1995),Drama|Sci-Fi,2.500195
2,1,32,4.0,1217896246,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,3.893225
3,1,47,4.0,1217896556,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,3.800833
4,1,50,4.0,1217896523,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,4.423616
...,...,...,...,...,...,...,...
105334,668,142488,4.0,1451535844,Spotlight (2015),Thriller,3.346001
105335,668,142507,3.5,1451535889,Pawn Sacrifice (2015),Drama,2.691712
105336,668,143385,4.0,1446388585,Bridge of Spies (2015),Drama|Thriller,2.998671
105337,668,144976,2.5,1448656898,Bone Tomahawk (2015),Horror|Western,2.453503
